In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Web Scraping

In [3]:
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [4]:
#The dataset contains top 1000 movies based on user rating
url  ='https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&ref_=adv_prv'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
response

<Response [200]>

In [6]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [8]:
#print(soup)
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
genre = []
director = []
actors = []

In [9]:
movie_data = soup.find_all('div', class_ = 'lister-item mode-advanced')

In [10]:
#Storing url of each page
for page in pages:
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start=' + str(page) + '&ref_=adv_nxt',headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    for i in movie_data:
        name = i.h3.a.text
        movie_name.append(name)
    
        yr = i.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(','').replace(')','')
        year.append(yr)
    
        duration = i.p.find('span', class_ = 'runtime').text.replace(' min','')
        time.append(duration)
    
        rate = i.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n','')
        rating.append(rate)
    
        meta = i.find('span', class_ = 'metascore').text if i.find('span', class_ = 'metascore') else '-'
        metascore.append(meta)
    
        value = i.find_all('span', attrs = {name :'nv'})
        vote = value[0].text if len(value) > 1 else '-'
        votes.append(vote)
    
        gross_amount = value[1].text if len(value) > 1 else '-'
        gross.append(gross_amount)
    
        gen = i.p.find('span', class_ = 'genre').text.replace('\n','')
        genre.append(gen)
        
        direc = i.find('p', class_ = '').find_all('a')[0].text
        director.append(direc)
        
        cast = [a.text for a in i.find('p', class_ = '').find_all('a')[1:]]
        actors.append(cast)


In [11]:
df = pd.DataFrame({
'Name of Movie': movie_name,
'Year of release': year,
'Runtime': time,
'Movie Rating': rating,
'Metascore': metascore,
'Votes': votes,
'Gross Collection': gross,
'Genre': genre,
'Director': director,
'Cast': actors
})

In [12]:
df

,Name of Movie,Year of release,Runtime,Movie Rating,Metascore,Votes,Gross Collection,Genre,Director,Cast
0,Jai Bhim,2021,164,9.5,-,-,-,"Crime, Drama, Mystery",T.J. Gnanavel,"[Suriya, Lijo Mol Jose, Manikandan K., Rajisha..."
1,Soorarai Pottru,2020,153,9.3,-,-,-,Drama,Sudha Kongara,"[Suriya, Paresh Rawal, Mohan Babu, Malini Sath..."
2,The Shawshank Redemption,1994,142,9.3,81,-,-,Drama,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
3,The Godfather,1972,175,9.2,100,-,-,"Crime, Drama",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
4,The Dark Knight,2008,152,9.1,84,-,-,"Action, Crime, Drama",Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
...,...,...,...,...,...,...,...,...,...,...
995,Raatchasan,2018,170,8.5,-,-,-,"Crime, Drama, Mystery",Ram Kumar,"[Vishnu Vishal, Amala Paul, Radha Ravi, Sangil..."
996,Parasite,2019,132,8.5,96,-,-,"Comedy, Drama, Thriller",Bong Joon Ho,"[Kang-ho Song, Sun-kyun Lee, Yeo-jeong Cho, Ch..."
997,Ayla: The Daughter of War,2017,125,8.5,-,-,-,"Biography, Drama, History",Can Ulkay,"[Halenur Uzunoglu, Çetin Tekindor, Ismail Haci..."
998,Avengers: Infinity War,2018,149,8.5,68,-,-,"Action, Adventure, Sci-Fi",Anthony Russo,"[Joe Russo, Robert Downey Jr., Chris Hemsworth..."


In [13]:
df.dtypes

Name of Movie       object
Year of release     object
Runtime             object
Movie Rating        object
Metascore           object
Votes               object
Gross Collection    object
Genre               object
Director            object
Cast                object
dtype: object

# Data Cleaning

In [14]:
df.dtypes

Name of Movie       object
Year of release     object
Runtime             object
Movie Rating        object
Metascore           object
Votes               object
Gross Collection    object
Genre               object
Director            object
Cast                object
dtype: object

In [15]:
df['Year of release'] = df['Year of release'].str.extract('(\d+)').astype(int)

In [16]:
df['Runtime'] = df['Runtime'].astype(int)

df['Movie Rating'] = df['Movie Rating'].astype(float)

df['Metascore'] = pd.to_numeric(df['Metascore'], errors = 'coerce')


In [17]:
df.dtypes

Name of Movie        object
Year of release       int32
Runtime               int32
Movie Rating        float64
Metascore           float64
Votes                object
Gross Collection     object
Genre                object
Director             object
Cast                 object
dtype: object

In [18]:
df['Genre'] = str(df['Genre'])

In [20]:
df['Name of Movie'] = str(df['Name of Movie'])
df['Director'] = str(df['Director'])
df['Cast'] = str(df['Cast'])